<a href="https://colab.research.google.com/github/Gilgamesh60/Techsoc_repo/blob/main/PS3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df=pd.read_csv("/content/drive/MyDrive/PS_3/PS3_train.csv")
df

,content,title,uid,target_ind
0,Premium quality five pocket jean from Wrangler...,Amazon.com: Wrangler Men's Rugged Wear Relaxed...,B0000CBALT,247
1,If you're looking for a different kind of anim...,Sakura Diaries - Complete Series Collector's E...,B00005QFDT,453
2,"First things first: Yes, Thinking XXX features...",Thinking XXX (Extended Cut) (2006),B000BNXD50,228
3,Feathertouch. 100% Polyester Machine Wash Warm...,Amazon.com: Petite Feathertouch Pull-On Pant: ...,B0002LK9V2,223
4,"When you need outstanding fuel delivery, easy ...",ACDelco EP386 Fuel Pump,B000C9PA54,312
...,...,...,...,...
35107,"SArah Walker, Anthony Rolfe Johnson, Jean Rigb...",Britten - Gloriana (1984),B000I2IUM0,473
35108,Buy Teenage Mutant Ninja Turtles 4: Turtles in...,Teenage Mutant Ninja Turtles IV: Turtles in Time,B000035Y3N,74
35109,A 10 movie collection of women action flicks. ...,Women Who Kick Butt 10 Movie Pack (2001),B00005B7BW,460
35110,This huarache sandal is designed for all-day c...,Softspots Women's Tela Leather Sandal,B0002KIFAK,158


# Preprocessing

In [3]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from collections import Counter
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn import preprocessing
nltk.download('wordnet')
nltk.download('omw-1.4')
ps = PorterStemmer()
wordnet_map = {"N":wordnet.NOUN, "V": wordnet.VERB, "J": wordnet.ADJ, "R": wordnet.ADV}
def lower_text(df,text):
    df[text] = df[text].str.lower()
    return df

def remove_punctuations(text):
    res = re.sub(r'[^\w\s]', '', text)
    return res

def remove_stopwords(text):
    return " ".join([word for word in text.split() if word not in stopwords.words('english')])


def find_freq_words(df,text):
    word_count = Counter()
    for text in df[text]:
        for word in text.split():
            word_count[word] += 1

    FREQUENT_WORDS = set(word for (word, wc) in word_count.most_common(3))
    return FREQUENT_WORDS

def remove_freq_words(text,FREQUENT_WORDS):
    return " ".join([word for word in text.split() if word not in FREQUENT_WORDS])

def stemming(text):
    return " ".join([ps.stem(word) for word in text.split()])

def remove_url(text):
    return re.sub(r'https?://\S+|www\.\S+', '', text)


def clean_chars(text):
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
df=lower_text(df,'content')
df['content']=df['content'].apply(lambda x : remove_punctuations(x))
df['content']=df['content'].apply(lambda x : remove_stopwords(x))
FREQUENT_WORDS=find_freq_words(df,'content')
df['content']=df['content'].apply(lambda x : remove_freq_words(x,FREQUENT_WORDS))
df['content']=df['content'].apply(lambda x : stemming(x))
df['content']=df['content'].apply(lambda x :  remove_url(x))
df['content']=df['content'].apply(lambda x: clean_chars(x))

In [5]:
df=lower_text(df,'title')
df['title']=df['title'].apply(lambda x : remove_punctuations(x))
df['title']=df['title'].apply(lambda x : remove_stopwords(x))
FREQUENT_WORDS=find_freq_words(df,'title')
df['title']=df['title'].apply(lambda x : remove_freq_words(x,FREQUENT_WORDS))
df['title']=df['title'].apply(lambda x : stemming(x))
df['title']=df['title'].apply(lambda x :  remove_url(x))
df['title']=df['title'].apply(lambda x: clean_chars(x))

# Model building


Tried out a bunch of different models.As expected passive aggresive classifier performed the best for text classification


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
df['data']=df['content']+df['title']
X = df['data']
y = df['target_ind']
# train test split (66% train - 33% test)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf= TfidfVectorizer()
X_train_cv = tf.fit_transform(X_train)
from sklearn.linear_model import PassiveAggressiveClassifier
PA = PassiveAggressiveClassifier(C = 0.3, random_state = 3)
PA.fit(X_train_cv, y_train)
X_test_cv = tf.transform(X_test)
predictions = PA.predict(X_test_cv)

In [18]:
from sklearn.metrics import r2_score
r2_score(predictions,y_test)

0.5452515002137712

# Testing

In [19]:
df_test=pd.read_csv('/content/drive/MyDrive/PS_3/PS3_test.csv')
df_test

,content,title,uid
0,Socket Cap Screws are reliable and durable and...,"Alloy Steel Socket Cap Screw, Flat Head, Hex S...",0
1,"This O-ring is made of black Buna-nitrile, is ...","-161 Buna O-Ring, 70A Durometer, Black, 5-1/2&...",1
2,"This Viton O-ring is black in color, round in ...","-445 Viton O-Ring, 75A Durometer, Black, 8&#03...",2
3,This 5-ounce lightweight pique knit sport shir...,Amazon.com: Port Authority Silk Touch Sport Sh...,3
4,"This O-ring is made of black Buna-nitrile, is ...","-359 Buna O-Ring, 70A Durometer, Black, 5-3/4&...",4
...,...,...,...
8101,"Gerald Mohr, Cathy O'Donnell. To help her get ...",Terror in the Haunted House,8101
8102,The first Go-Kart Racing game to come out for ...,Super Mario Kart,8102
8103,For almost two decades--the '60s and the '70s-...,Smokey Robinson - Standing Room Only (1990),8103
8104,Patrick Murray's Reflections on Commercial Lif...,Reflections on Commercial Life: An Anthology o...,8104


In [20]:
df=df_test
df=lower_text(df,'content')
df['content']=df['content'].apply(lambda x : remove_punctuations(x))
df['content']=df['content'].apply(lambda x : remove_stopwords(x))
FREQUENT_WORDS=find_freq_words(df,'content')
df['content']=df['content'].apply(lambda x : remove_freq_words(x,FREQUENT_WORDS))
df['content']=df['content'].apply(lambda x : stemming(x))
df['content']=df['content'].apply(lambda x :  remove_url(x))
df['content']=df['content'].apply(lambda x: clean_chars(x))

df=lower_text(df,'title')
df['title']=df['title'].apply(lambda x : remove_punctuations(x))
df['title']=df['title'].apply(lambda x : remove_stopwords(x))
FREQUENT_WORDS=find_freq_words(df,'title')
df['title']=df['title'].apply(lambda x : remove_freq_words(x,FREQUENT_WORDS))
df['title']=df['title'].apply(lambda x : stemming(x))
df['title']=df['title'].apply(lambda x :  remove_url(x))
df['title']=df['title'].apply(lambda x: clean_chars(x))

df['data']=df['content']+df['title']
X = df['data']
tf= TfidfVectorizer()
X_cv= tf.fit_transform(X)

In [26]:
X_test = tf.transform(X)

In [28]:
predictions = PA.predict(X_test)

In [29]:
predictions

array([361, 351, 349, ..., 431,   9, 453])

In [ ]:
df['target_ind']=pd.DataFrame(predictions)

In [32]:
df=df[['uid','target_ind']]
df.to_csv('submission.csv',index=False)